## Мотивација и методолошки пристап

Целта на овој проект е да се изгради модел кој ќе предвидува дали заемопримачот ќе го врати кредитот (`loan_paid_back`). Овој проблем не е само технички, туку и бизнис-критичен, бидејќи различните типови на грешки носат различен ризик. Во реален кредитен систем, многу е поопасно да се одобри кредит на лице кое нема да го врати, отколку да се одбие кредит на лице кое реално би го вратило. Поради тоа, во овој проект се применува cost-sensitive пристап, каде што лажно позитивните предвидувања (false positives) се сметаат за пет пати поскапи од лажно негативните (false negatives).

Податоците содржат нумеричккатегорискиални карактеристики, а различните модели имаат различни барања во однос на начинот на нивна обработка. Моделите како Logistic Regression и Multilayer Perceptron се чувствителни на размерите на вредностите и бараат стандардизација на нумеричките податоци за да функционираат стабилно и ефикасно. Од друга страна, tree-based моделите како Random Forest, XGBoost и LightGBM не зависат од скалата на карактеристиките на ист начин, бидејќи одлуките ги носат преку поделби (splits) во дрва. Поради тоа се користат две различни preprocessing pipeline-и: еден со стандардизација за модели чувствителни на скала и еден без стандардизација за tree-based модели, при што во двата случаи категоријалните променливи се претвораат во нумерички преку One-Hot Encoding.

Наместо да се користи стандарден праг од 0.5 за носење одлука, секој модел се евалуира преку оптимизација на прагот врз основа на дефинираната cost функција. Ова значи дека одлуката дали еден заем ќе биде одобрен или не не се базира само на веројатноста што ја дава моделот, туку и на реалниот ризик поврзан со можната грешка. Како резултат на ваквиот пристап, очекувано е моделите да бидат поконзервативни и да користат високи прагови за позитивна класа, со цел да се минимизира бројот на ризични одобрувања.

Покрај поединечните модели, изграден е и ансамбл модел преку soft voting, каде што се комбинираат веројатностите од сите модели. Мотивацијата за ансамблот е дека различните модели можат да научат различни шеми и релации во податоците, а нивната комбинација може да доведе до постабилни и поробустни предвидувања. Сепак, ансамблот не се третира како автоматски подобро решение, туку како дополнителен чекор кој се евалуира под истите cost-sensitive критериуми како и останатите модели.


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv("loan_dataset_20000.csv") 

X_raw = df.drop("loan_paid_back", axis=1)
y = df["loan_paid_back"]

# monthly and yearly income carry the same info monhly = yearly / 12
X_raw = X_raw.drop(columns=['monthly_income'])


In [4]:
X = X_raw.copy()
y = y.copy()

categorical_cols = X.select_dtypes(include='object').columns.tolist()
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline


preprocessor_scaled = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

preprocessor_tree = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)


In [6]:
from sklearn.model_selection import train_test_split


target_col = "loan_paid_back" 
X = df.drop(columns=[target_col])
y = df[target_col]

# stratify keeps the 0/1 ratio similar in train and test (important for FP/FN work)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes: ", X_test.shape, y_test.shape)
print("\nClass distribution (train):\n", y_train.value_counts(normalize=True))
print("\nClass distribution (test):\n", y_test.value_counts(normalize=True))


Train shapes: (16000, 21) (16000,)
Test shapes:  (4000, 21) (4000,)

Class distribution (train):
 loan_paid_back
1    0.799875
0    0.200125
Name: proportion, dtype: float64

Class distribution (test):
 loan_paid_back
1    0.8
0    0.2
Name: proportion, dtype: float64


### Helper functions ###

In [7]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight

COST_FP = 5
COST_FN = 1

def cost_and_cm(y_true, y_pred, cost_fp=5, cost_fn=1):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    cost = cost_fp * fp + cost_fn * fn
    return cost, (tn, fp, fn, tp)

def best_threshold_by_cost(y_true, proba_pos, cost_fp=5, cost_fn=1):
    thresholds = np.linspace(0.01, 0.99, 199)
    best = {"threshold": 0.5, "cost": float("inf"), "cm": None}

    for t in thresholds:
        y_pred = (proba_pos >= t).astype(int)
        cost, cm = cost_and_cm(y_true, y_pred, cost_fp, cost_fn)
        if cost < best["cost"]:
            best = {"threshold": float(t), "cost": float(cost), "cm": cm}
    return best

def print_best(name, best):
    tn, fp, fn, tp = best["cm"]
    print(f"{name} | best threshold = {best['threshold']:.3f} | cost = {best['cost']:.0f}")
    print(f"{name} | confusion (tn, fp, fn, tp) = {tn}, {fp}, {fn}, {tp}")


### Logistic regression ###

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

lr_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_scaled),
    ("model", LogisticRegression(
        max_iter=5000,
        class_weight={0: 1, 1: COST_FP}  # pushes toward fewer FP at a given threshold
    ))
])

# optional sample weights (extra cost-sensitivity)
sw = compute_sample_weight(class_weight={0: 1, 1: COST_FP}, y=y_train)

# LR supports sample_weight
lr_pipe.fit(X_train, y_train, model__sample_weight=sw)

p_lr = lr_pipe.predict_proba(X_test)[:, 1]
best_lr = best_threshold_by_cost(y_test, p_lr, COST_FP, COST_FN)
print_best("LogReg", best_lr)


LogReg | best threshold = 0.990 | cost = 1626
LogReg | confusion (tn, fp, fn, tp) = 564, 236, 446, 2754


### MLP ###

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

mlp_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_scaled),
    ("model", MLPClassifier(
        hidden_layer_sizes=(64, 32),
        max_iter=500,
        random_state=42,
        early_stopping=True
    ))
])

sw = compute_sample_weight(class_weight={0: 1, 1: COST_FP}, y=y_train)

# MLPClassifier supports sample_weight in sklearn versions that are reasonably recent.
# If yours errors, remove model__sample_weight and rely on thresholding.
try:
    mlp_pipe.fit(X_train, y_train, model__sample_weight=sw)
    used_sw = True
except TypeError:
    mlp_pipe.fit(X_train, y_train)
    used_sw = False

p_mlp = mlp_pipe.predict_proba(X_test)[:, 1]
best_mlp = best_threshold_by_cost(y_test, p_mlp, COST_FP, COST_FN)
print("MLP used sample_weight:", used_sw)
print_best("MLP", best_mlp)


MLP used sample_weight: True
MLP | best threshold = 0.970 | cost = 1613
MLP | confusion (tn, fp, fn, tp) = 654, 146, 883, 2317


### Random forrest ###

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

rf_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_tree),
    ("model", RandomForestClassifier(
        n_estimators=600,
        random_state=42,
        n_jobs=-1,
        class_weight={0: 1, 1: COST_FP}
    ))
])

sw = compute_sample_weight(class_weight={0: 1, 1: COST_FP}, y=y_train)

# RF supports sample_weight
rf_pipe.fit(X_train, y_train, model__sample_weight=sw)

p_rf = rf_pipe.predict_proba(X_test)[:, 1]
best_rf = best_threshold_by_cost(y_test, p_rf, COST_FP, COST_FN)
print_best("RandomForest", best_rf)


RandomForest | best threshold = 0.752 | cost = 1674
RandomForest | confusion (tn, fp, fn, tp) = 542, 258, 384, 2816


### XGBoost ###

In [11]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

xgb_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_tree),
    ("model", XGBClassifier(
        n_estimators=800,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        random_state=42,
        eval_metric="logloss",
        n_jobs=-1
    ))
])

sw = compute_sample_weight(class_weight={0: 1, 1: COST_FP}, y=y_train)

# XGB supports sample_weight
xgb_pipe.fit(X_train, y_train, model__sample_weight=sw)

p_xgb = xgb_pipe.predict_proba(X_test)[:, 1]
best_xgb = best_threshold_by_cost(y_test, p_xgb, COST_FP, COST_FN)
print_best("XGBoost", best_xgb)


XGBoost | best threshold = 0.945 | cost = 1587
XGBoost | confusion (tn, fp, fn, tp) = 576, 224, 467, 2733


### LightGBM ###

In [12]:
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline

lgbm_pipe = Pipeline(steps=[
    ("preprocess", preprocessor_tree),
    ("model", LGBMClassifier(
        n_estimators=1200,
        learning_rate=0.03,
        num_leaves=31,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
        class_weight={0: 1, 1: COST_FP}
    ))
])

sw = compute_sample_weight(class_weight={0: 1, 1: COST_FP}, y=y_train)

# LGBM supports sample_weight
lgbm_pipe.fit(X_train, y_train, model__sample_weight=sw)

p_lgbm = lgbm_pipe.predict_proba(X_test)[:, 1]
best_lgbm = best_threshold_by_cost(y_test, p_lgbm, COST_FP, COST_FN)
print_best("LightGBM", best_lgbm)


[LightGBM] [Info] Number of positive: 12798, number of negative: 3202
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000539 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2251
[LightGBM] [Info] Number of data points in the train set: 16000, number of used features: 69
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.990091 -> initscore=4.604389
[LightGBM] [Info] Start training from score 4.604389


C:\Users\jakim\miniconda3\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


LightGBM | best threshold = 0.990 | cost = 1644
LightGBM | confusion (tn, fp, fn, tp) = 609, 191, 689, 2511


### Soft voting ensemble ###

In [13]:
# Equal-weight soft vote
p_ens = (p_lr + p_mlp + p_rf + p_xgb + p_lgbm) / 5.0

best_ens = best_threshold_by_cost(y_test, p_ens, COST_FP, COST_FN)
print_best("Ensemble(soft)", best_ens)


Ensemble(soft) | best threshold = 0.945 | cost = 1616
Ensemble(soft) | confusion (tn, fp, fn, tp) = 631, 169, 771, 2429
